# TP Pytest

<center>
<span style="font-style: italic">Loic Gouarin</span>
</center>
<center>
<span>du 1 au 3 octobre 2018</span>
</center>

Vous avez à présent une connaissance des outils qui peuvent vous être utiles pour écrire des codes Python de qualité et les packager. Si vous deviez exécuter ces outils à la main à chaque fois que vous faites un changement, vous abandonneriez vite tellement la tâche vous paraîtrait fastidieuse. 

Dans ce dernier TP, nous allons donc regarder comment automatiser toutes ces tâches en se servant de github. Par conséquent, la première chose à faire est de mettre le code sur github.

# Github

Si vous n'avez pas encore de compte sur github, créez en un [ici](https://github.com/join?source=header). Vous devez ensuite créer un nouveau projet que vous pouvez par exemple nommer `splinart`.

L'idée étant de mettre dans ce dépôt le `final_step` de la partie sur `pytest`, copiez ce répertoire dans un répertoire à part. Lorsque vous créez un dépôt sur github, celui-ci vous donne plusieurs recettes pour l'initialiser. Nous choisirons la première. 

Mais avant cela, nous allons ajouter un fichier `readme.rst` et un `.gitignore`.

## readme.rst

```rst
splinart is a package used for a tutorial which explains how to do the Python packaging using

- `PyPi <https://pypi.python.org/pypi>`_
- `conda build <https://conda.io/docs/user-guide/tasks/build-packages/recipe.html>`_
- `pytest <https://docs.pytest.org/en/latest/>`_
- `Pylint <https://www.pylint.org/>`_
- `Sphinx <http://www.sphinx-doc.org/en/stable/>`_

And automate the process to distribute this package using github.

The original idea of splinart is found on the great invonvergent website.

If you want to install splinart::

    pip install splpinart

or::

    conda install -c gouarin splinart
```

## .gitignore

```
build
_build
.cache
dist
.ipynb_checkpoints
__pycache__
```

Vous pouvez à présent initialiser votre projet en utilisant la commande suivante

    git init
    
Nous allons dans un premier temps ajouter les deux fichiers que nous avons créés précédemment.

    git add readme.rst .gitignore
    git commit -m "initial commit"
   
Puis tous les autres fichiers en faisant

    git add splinart doc demos ...

Vérifiez avant de faire le commit que tous les fichiers sont les bons et qu'il n'en manque pas.

    git commit -m "add splinart"

Initialisez le `remote` en suivant ce qui est écrit sur votre github. J'avais pour cette exemple

    git remote add origin https://github.com/gouarin/splinart.git
    
Vous pouvez maintenant faire un push de vos trois commits en faisant

    git push --set-upstream origin master

Vous pouvez à présent vérifier que tous vos fichiers sont présents sur votre github.

Maintenant que notre dépôt a bien été créé, nous allons nous intéresser (dans l'ordre) aux étapes suivantes

- Mise en place de travis pour l'intégration continue.
- Validation des push en utilisant pytest et pylint.
- Génération de la documentation sur ReadTheDocs.
- Déploiement de `splinart` sur PyPi et conda.
    
# Travis

[Travis CI](https://travis-ci.org/) vous permet de faire de l'intégration continue sur votre projet et de l'utiliser comme un plugin de github dans un evironnement linux ou mac os. L'idée étant de tester son projet sur différents OS et différentes versions de Python afin de s'assurer du bon comportement de celui-ci. La première chose à faire est de se connecter à travis en utilisant son compte github.

Puis d'aller sur son compte en haut à droite de la fenêtre. Vous verrez alors vos projets github. Il vous suffit à présent d'activer celui appelé `splinart` (si vous l'avez appelé comme ça).

Travis fonctionne en lisant un fichier `.travis.yml` décrivant les logiciels nécessaires pour installer notre package et la façon de le tester. Cette étape nous permet de valider une mise à jour du dépôt réalisée sous forme d'un pull request ou d'un merge request. De cette manière, nous sommes plus confiant dans les changements.

Nous allons commencer par ajouter un fichier `.travis.yml` très simple afin de comprendre un peu mieux comment ça se passe.

In [ ]:
%%file .travis.yml

language: python

install:
  - sudo apt-get update
  - wget https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh -O miniconda.sh
  - bash miniconda.sh -b -p $HOME/miniconda
  - export PATH="$HOME/miniconda/bin:$PATH"
  - hash -r
  - conda config --set always_yes yes --set changeps1 no
  - conda update -q conda
  # Useful for debugging any issues with conda
  - conda info -a

  - conda create -q -n splinart-env python=$TRAVIS_PYTHON_VERSION
  - source activate splinart-env
  - conda install numpy matplotlib six pytest
  - python setup.py install

script:
  # Your test script goes here
  - pytest tests

Nous voyons trois parties dans ce fichier

- `language` indique le langage que nous allons utiliser et sa version. Mais vu que nous allons installer un miniconda, celui-ci ne sert pas vraiment sauf pour le flag `$TRAVIS_PYTHON_VERSION`. Travis a un ensemble de flags qui nous facilite les choses (https://docs.travis-ci.com/user/reference/overview/).
- `install` est la section qui dit ce qu'il faut faire pour installer notre package. Vous pouvez voir que ce sont des commandes unix et qu'elles correspondent exactement à ce que vous pourriez faire vous même sur votre système.
- `script` indique ce que l'on fait une fois que l'on a fini d'installer. Nous ferons pour le moment les tests. Si les tests échouent, travis nous dira que la tentative de construction de notre projet a échoué.

Ajoutez et poussez ce fichier sur votre dépôt github. Vous devriez voir tavis réagir et réussir à construire le projet.

Dans ce que nous avons fait, nous voyons que nous avons installé en dur dans le fichier numpy, matplotlib et six. Il se peut que nous ayons d'autres dépendances à l'avenir et nous voudrions que ce soit un peu plus explicite. Nous allons donc créer un fichier `requirements.txt` qui indique ces dépendances.

In [1]:
%%file requirements.txt
numpy
matplotlib
six
pytest

Writing requirements.txt


Le fichier `.travis.yml` devient

In [2]:
%%file .travis.yml

language: python

install:
  - sudo apt-get update
  - wget https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh -O miniconda.sh
  - bash miniconda.sh -b -p $HOME/miniconda
  - export PATH="$HOME/miniconda/bin:$PATH"
  - hash -r
  - conda config --set always_yes yes --set changeps1 no
  - conda update -q conda
  # Useful for debugging any issues with conda
  - conda info -a

  - conda create -q -n splinart-env python=$TRAVIS_PYTHON_VERSION
  - source activate splinart-env
  - travis_wait pip install -r requirements.txt
  - python setup.py install

script:
  # Your test script goes here
  - pytest tests

Writing .travis.yml


`travis_wait` indique à travis d'attendre si la procédure est un peu longue. 

Mettez à jour votre dépôt en ajoutant `requirements.txt` et `.travis.yml`. Vérifiez que la construction du projet se passe bien.

Nous allons à présent ajouter `pylint` et le test de couverture du projet. Pour cela, nous allons ajouter dans `requirements.txt` les packages suivants

In [ ]:
pylint
pytest-pylint
pytest-cov
codecov

Puis modifier le fichier `.travis.yml` en conséquence en modifiant juste au niveau de la section `script`

In [ ]:
script:
  - pytest --pylint --pylint-rcfile=.pyrcfile --pylint-error-types=EF splinart
  - pytest --cov=splinart tests

after_success:
  - codecov  -e $TRAVIS_PYTHON_VERSION

Nous indiquons ici à `pytest` que les types d'erreurs `E` et `F` de pylint sont considérés comme des erreurs.

Nous avons à présent un script qui fonctionne bien pour une certaine version de Python. Nous aimerions maintenant tester notre package pour différentes version de Python sur linux et mac os. Travis permet de créer des `matrix` indiquant un certain nombre de builds différents sans écrire tous les scripts correspondant à la main.

Nous allons donc lui demander de tester notre projet pour différentes versions de Python sur linux et mac os.

In [3]:
%%file .travis.yml

language: python
env:
  - PYVER="2.7"
  - PYVER="3.4"
  - PYVER="3.5"
  - PYVER="3.6"
os:
  - linux
  - osx

before_install:
  - |
    if [[ "$TRAVIS_OS_NAME" == "osx" ]]; then 
      wget https://repo.continuum.io/miniconda/Miniconda3-latest-MacOSX-x86_64.sh -O miniconda.sh;
      mkdir .matplotlib;
      echo "backend : TkAgg" >> .matplotlib/matplotlibrc
    fi
  - |
    if [[ "$TRAVIS_OS_NAME" == "linux" ]]; then 
      sudo apt-get update;
      wget https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh -O miniconda.sh;
    fi
install:
  - bash miniconda.sh -b -p $HOME/miniconda
  - export PATH="$HOME/miniconda/bin:$PATH"
  - hash -r
  - conda config --set always_yes yes --set changeps1 no
  - conda update -q conda
  # Useful for debugging any issues with conda
  - conda info -a
  - conda create -q -n splinart-env python=$PYVER
  - source activate splinart-env
  - travis_wait pip install -r requirements.txt
  - if [[ "$TRAVIS_OS_NAME" == "osx" ]]; then conda install python.app; fi;
  - python setup.py install

script:
  - pytest --pylint --pylint-rcfile=.pyrcfile --pylint-error-types=EF splinart
  - pytest --cov=splinart tests

after_success:
  - codecov -e $PYVER

Overwriting .travis.yml


La variable d'environnement `TRAVIS_OS_NAME` permet de savoir si nous sommes sur linux ou sur mac osx. Nous installons donc différemment miniconda en fonction du système d'exploitation. Il y a un problème de backend avec mac osx, c'est pourquoi nous ajoutons un fichier `matplotlibrc`.

Nous avons donc pu tester notre projet sur

| linux |  macosx |
|:-----:|:-------:|
| python 2.7 | python 2.7|
| python 3.4 | python 3.4|
| python 3.5 | python 3.5|
| python 3.6 | python 3.6|

## ReadTheDocs

Nous allons à présent nous intéresser à générer la documentation automatiquement sur ReadTheDocs. Pour cela, nous allons ajouter un fichier `environment.yml` dans le répertoire `doc` indiquant tout ce dont nous avons besoin pour sphinx et ses dépendances. Et un fichier `.readthedocs.yml` à la racine du projet indiquant comment installer notre projet.

Voici à quoi ressemblent ces deux fichiers

In [4]:
%%file environment.yml
name: splinart
dependencies:
  - notebook
  - sphinx
  - nbsphinx

Writing environment.yml


In [5]:
%%file .readthedocs.yml
conda:
   file: doc/environment.yml
requirements_file: requirements.txt
python:
   setup_py_install: true

Writing .readthedocs.yml


Vous devez à présent vous créer un compte sur https://readthedocs.org/ et connecter votre projet github à celui-ci. De cette manière, à chaque fois que vous ferez une mise à jour du dépôt, la documentation sera regénérée.

## Déploiement sur PyPi et conda

Nous voudrions maintenant que lorsque l'on met un tag sur github, la version soit automatiquement mise sur `PyPi` et sur `anaconda`. Pour ce faire, il faut autoriser travis à accéder à votre compte `Pypi` et à votre compte `anaconda` et de manière sécurisée (pas de mots de passe en clair !!).

Il faut tout d'abord créer un token sur votre compte `anaconda` en allant dans la partie `access`. Cette accès devra avoir les droits suivants

- `api:read` (Allow read access to the API site)
- `api:write` (Allow write access to the API site)

Il faut ensuite installer `travis` en local et générer les clés cryptées (pour plus d'informations https://docs.travis-ci.com/user/encryption-keys/).

- Installer `travis` à l'aide de la commande

    gem install travis

- Aller dans le projet où se trouve le fichier `.travis.yml`

- Générer la clé cryptée pour votre mot de passe `PyPi`

    travis encrypt PYPI_PASSWD=votre_mot_de_passe_pypi --add
    
- Générer la clé cryptée pour le token `anaconda`

    travis encrypt ANACONDA_TOKEN=votre_token_anconda --add

**Attention**: les noms choisis (`PYPI_PASSWD` et `ANACONDA_TOKEN`) ont leur importance.

Vous devriez voir au début de votre fichier `.travis.yml` des lignes qui ressemblent à ça

```
 - secure: qZdQddtVkVpF/60at7mwQiIr7xVK5inETXwGZdCGg0wKeM9Qp0Ae7z6UjGOE+WLNt+QNCcR1tR4BVKlLZmvviMZFPyifdVpKs8GBsx77abaVWfAGOUtaP8HyLFA5/GEW+CJkrZnPWLiumMRE+XIfj6cH4iMBWkKGvHJ+p7YcOQJOUmqXv1S2fOP72bnRmQhwPXk807wNLumDf/MgnXTHx2twtQ/a6EV+z21fKbc8AzP7yoztoxyVw3AdmwOekFkQ+sT9t3AphptwOpuk9Kl18oUbhiDMGhIFdOFgwJS+yfc1cBycro8l50IQyKFYjKPdKrHY32TW6IvPPi7r4q6lEoCUMgJd42CuQrcLfundSoubMBgkxlc3Cv3cbAIzHrpav6yCWbOuiV/OEcVK+JMg5BXqwweOEvcnQq8SQWoK7A5YSXt7h4P5LNdelDl4ZNEs37L3SKsHdnfyXtlEWhFttd8gNYBJ7K9Fx3bF4tocUr5OFXkAMSW/T5jeH7sxJRh+eoAkbJJaxHkPpVnNDxX/U3W5KfwF9kiFEgOsSRuCcDmrTLBVY8Nt4Vy/z4lO+U4Mk7L/esUb1RCcNzVcBsh1l3PA5Hs/my02Qj9natcmORbv+htneKXXBZwCbohEQLSvqm148BCpGPI+eGozCGcc5MljQdZm1x/b6KiG2gg8C9g=
```

Vous pouvez maintenant mettre à jour les dépôts `PyPi` et `anaconda`. Il nous reste à dire à `travis` comment le faire et à quel moment (lorsque nous créons un tag dans notre dépôt git).

Nous allons tout d'abord dire à `travis` que, si la version qu'il teste est taguée via la variable `$TRAVIS_TAG` alors il construit le package `conda` et la distribution `sdist` pour `PyPi`. Pour ce faire, il vous suffit de rajouter les lignes suivantes dans la partie `script`.

```
- |
  if [[ $TRAVIS_TAG ]]; then
    python setup.py sdist;
    conda build -q --python ${PYVER} recipes &&
    conda install --use-local ${PROJECT_NAME} &&
    conda uninstall ${PROJECT_NAME};
  fi
```

Vous pouvez voir, qu'en plus de construire le package `conda`, nous demandons à `travis` de l'installer en local puis de le désinstaller.

`travis` a également une partie `deploy` qui indique comment déployer le projet si tout s'est bien passé. Voici ce que ça donne

```
deploy:
- provider: pypi
  skip_cleanup: true
  user: gouarin
  password: "${PYPI_PASSWD}"
  server: https://test.pypi.org/legacy/
  on:
    repo: "${GITHUB_REPO_NAME}"
    tags: true
    condition: "$PYVER == 2.7 && $TRAVIS_OS_NAME == linux"
- provider: script
  skip_cleanup: true
  script: anaconda -t $ANACONDA_TOKEN upload --force ${HOME}/miniconda/conda-bld/*/${PROJECT_NAME}-*.tar.bz2
  on:
    repo: "${GITHUB_REPO_NAME}"
    tags: true
```

Il faut remarquer que le déploiement se fera que si il y a un tag sur le dépôt `github`

```
  on:
    repo: "${GITHUB_REPO_NAME}"
    tags: true
```

Nous avons ajouté deux variables d'environnement au début du fichier de la façon suivante

```
env:
  global:
  - PROJECT_NAME=splinart
  - GITHUB_REPO_NAME=gouarin/splinart
```

Il ne vous suffit plus qu'à mettre ce nouveau fichier sur le dépôt, puis de mettre un tag en tapant les commandes suivantes si vous voulez par exemple mettre une version `0.1.0`

```
git tag 0.1.0
git push --tags
```

`travis` s'occupe du reste !!

**Remarque**: Nous ne vérifions pas que le tag donné correspond à la version du projet se trouvant dans le fichier `setup.py`. Il faudra le vérifier.

## Appveyor

Nous avons à présent une version sur `PyPi` qui fonctionne pour n'importe quel système d'exploitation puisque c'est un package écrit en pur Python et une version pour linux et mac osx sur `conda`. Ca serait bien d'avoir également une version que l'on pourrait installer sous windows !! Et c'est là que `appveyor` entre en jeu. C'est le système d'intégration continue que propose `github` pour travailler sur des machines windows.

Nous allons passer les détails car la procédure est la même que pour `travis`. Il vous faut donc là encore connecter votre projet `github` à `appveyor` en allant ici https://www.appveyor.com/.

Là encore, il faut crypter son token anaconda en se rendant sur cette page

https://ci.appveyor.com/tools/encrypt

**Attention** il ne faut mettre que le token et non `ANACONDA_TOKEN=xxxx`.

Il vous reste à créer un fichier `.appveyor` contenant les informations suivantes

In [ ]:
%%file .appveyor

# Configure appveyor for builds.

environment:
  condatoken:
    secure: IgQ0ihtOWOGDj+VRskpoImtpJQaF9LAAwD0rA9/xQXa7fIr3N8+hQM8aAMi/zXC/

  # Need this to set up compilation on Windows.
  CMD_IN_ENV: cmd /E:ON /V:ON /C Obvious-CI\scripts\obvci_appveyor_python_build_env.cmd
  
  PROJECT_NAME: splinart
  
  matrix:
    # Unfortunately, compiler/SDK configuration for 64 bit builds depends on
    # python version. Right now conda build does not configure the SDK, and
    # the appveyor setup only sets up the SDK once, so separate by python
    # versions.
    - TARGET_ARCH: "x64"
      PYTHON_BUILD_RESTRICTIONS: "2.7*"
      CONDA_PY: "27"
      CONDA_INSTALL_LOCN: "C:\\Miniconda-x64"
      CONDA_BUILDS:  C:\\Miniconda-x64\conda-bld\win-64
    - TARGET_ARCH: "x64"
      PYTHON_BUILD_RESTRICTIONS: "3.5*"
      CONDA_PY: "35"
      CONDA_INSTALL_LOCN: "C:\\Miniconda35-x64"
      CONDA_BUILDS:  C:\\Miniconda35-x64\conda-bld\win-64
    - TARGET_ARCH: "x64"
      PYTHON_BUILD_RESTRICTIONS: "3.6*"
      CONDA_PY: "36"
      CONDA_INSTALL_LOCN: "C:\\Miniconda36-x64"
      CONDA_BUILDS:  C:\\Miniconda36-x64\conda-bld\win-64

# We always use a 64-bit machine, but can build x86 distributions
# with the TARGET_ARCH variable.
platform:
    - x64

install:
    # Clone simply to get the script for setting up Windows build environment.
    - cmd: git clone https://github.com/pelson/Obvious-CI.git

    # No need to install miniconda because appveyor comes with it.
    - cmd: SET PATH=%CONDA_INSTALL_LOCN%;%CONDA_INSTALL_LOCN%\Scripts;%PATH%

    - cmd: conda config --set always_yes true
    - cmd: conda update --quiet conda

    - cmd: conda install --quiet jinja2 conda-build=2 anaconda-client
    # These installs are needed on windows but not other platforms.
    - cmd: conda install patch psutil
    - cmd: pip install -r requirements.txt

# Skip .NET project specific build phase.
build: off

test_script:
    - "%CMD_IN_ENV% conda build recipes"

    # Install the package
    - conda install --use-local %PROJECT_NAME%

    # Run the tests outside the source tree.
    - pytest --cov %PROJECT_NAME% tests/

on_success:
    - echo %APPVEYOR_REPO_TAG%
    - cmd: SET PATH=%CONDA_INSTALL_LOCN%;%CONDA_INSTALL_LOCN%\Scripts;%PATH%
    # Write the output file location to a file...cannot simply use conda build --output
    # because astropy_helpers prints out a message when it is freezing the version number during setup.
    - python -c "from conda_build.api import get_output_file_path; p=get_output_file_path('recipes'); f=open('to_upload.txt', 'w');f.write(p)"
    # ...so that we can set a variable to the name of that output file.
    - set /P BUILT_PACKAGE=<to_upload.txt
    - echo %BUILT_PACKAGE%
    # If this build is because of a tag make the conda package and upload it.
    #- cmd: if "%APPVEYOR_REPO_TAG%"=="true" anaconda -t %BINSTAR_TOKEN% upload -u astropy %BUILT_PACKAGE%    
    - cmd: anaconda -t %condatoken% upload --force %BUILT_PACKAGE%


## Les badges

Il est possible de mettre tout un tas de badges dans votre fichier `readme`. Ceux-ci permettent de voir tout de suite un ensemble d'informations qui peuvent donner envie à un utilisateur potentiel de tester votre projet. Nous allons ajouter les badges suivants

- badge pour le package conda
- badge pour la documentation
- badge pour la construction du projet sur travis
- badge pour la construction du projet sur appveyor
- badge pour la couverture

La plupart des outils que vous avez utilisés donne sur la page de votre projet le lien à mettre pour insérer le badge. Par exemple, pour `travis`, il suffit d'ajouter à la fin du `readme`

```
.. |travis| image:: https://travis-ci.org/gouarin/splinart.svg?branch=master
   :target: https://travis-ci.org/gouarin/splinart
```

et au début

```
|travis|
```

ce qui permet d'ajouter le badge.

## La petite touche final: [binder](https://mybinder.org/)

`MyBinder` permet de créer un environnement prêt à l'emploi avec vos notebooks. A partir de là, vous pouvez dire à `MyBinder` de créer cet environnement avec votre projet (puisqu'il est maintenant sur `PyPi` et `conda`) et d'ouvrir le répertoire où il y a les notebooks de tutoriels. 

Un utilisateur peut alors tester votre projet rapidement sans rien installer sur son système !!!

Nous allons juste créer un fichier `environment.yml` indiquant à `binder` ce que l'on doit installer.

In [1]:
%%file environment.yml
name: splinart
channels:
  - gouarin
dependencies:
  - python=3.6
  - splinart

Writing environment.yml


Il suffit maintenant de copier-coller l'adresse de votre projet `github` dans l'interface de `binder` et de faire `launch`.

Vous n'avez plus qu'à copier le badge dans votre `readme` et n'importe qui peut tester votre projet rapidement et facilement !